In [ ]:
def execute_python_script():
    import pandas as pd
    from datetime import datetime

    try:
        print("Ejecutando el script Python...")
        file_path = "/content/Hoja de cálculo en Basis (1).xlsx"
        data = pd.read_excel(file_path)
        print("Columnas en el DataFrame:", data.columns.tolist())

        # Limpieza y formateo inicial de datos
        data.columns = data.columns.str.strip()
        data['CENTRO'] = data['Centro'].astype(str).str.extract('(\d+)')
        data['MATERIAL'] = data['Material'].astype(str).str.extract('(\d+)')
        ## aqui corregui los caracteres para que me genere todos los datos de DESCRIPCION
        data['DESCRIPCION'] = data['Material'].astype(str).str.extract('([A-Za-z].*)')
        data['MONEDA'] = 'COP'
        data['ALMACEN'] = data['Almacén'].astype(str).str.extract('(\d+)')[0].str[4:]

        data.rename(columns={
            'CtdStk SV': 'CANTIDAD_STOCK',
            'ValStkVal': 'VALOR_STOCK'
        }, inplace=True)

        data['CANTIDAD_STOCK'] = pd.to_numeric(data['CANTIDAD_STOCK'], errors='coerce')

        execution_date = datetime.now().strftime('%Y-%m-%d')
        data['FECHA_EJECUCION'] = execution_date

        data_aggregated = data.groupby('MATERIAL').agg({
            'CENTRO': lambda x: ', '.join(set(x.dropna())),
            'DESCRIPCION': lambda x: ', '.join(set(x.dropna())),
            'ALMACEN': lambda x: ', '.join(set(x.dropna())),
            'CANTIDAD_STOCK': 'sum',
            'VALOR_STOCK': 'sum',
            'FECHA_EJECUCION': 'max'
        }).reset_index()

        data_aggregated.rename(columns={
            'CENTRO': 'CENTROS',
            'DESCRIPCION': 'DESCRIPCIONES',
            'ALMACEN': 'ALMACENES',
            'CANTIDAD_STOCK': 'CANTIDAD_STOCK_total',
            'VALOR_STOCK': 'VALOR_STOCK_total'
        }, inplace=True)

        #  VALOR_STOCK_total a string antes de realizar el reemplazo
        data_aggregated['VALOR_STOCK_total'] = data_aggregated['VALOR_STOCK_total'].apply(lambda x: f'{x:,.2f}')

        # comas por puntos y puntos por comas en 'VALOR_STOCK_total
        data_aggregated['VALOR_STOCK_total'] = data_aggregated['VALOR_STOCK_total'].str.replace('.', 'temp').str.replace(',', '.').str.replace('temp', ',')

        # Here I agg the column "MONEDA"
        data_aggregated['MONEDA'] = 'COP'

        output_file_path = f"/content/datos/archivo_modificado_integrado_{execution_date}.xlsx"
        data_aggregated.to_excel(output_file_path, index=False)
        print("El archivo ha sido guardado exitosamente como:", output_file_path)
    except Exception as e:
        print(f"Error al ejecutar el script Python: {e}")

execute_python_script()
